# <center>Deep Learning в РС. Tensorflow

In [6]:
import pandas  as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

In [5]:
df = pd.read_csv('data/ratings.csv')
df.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [8]:
X_train, X_test = train_test_split(df, random_state=42, test_size=0.2)
X_train.shape[0], X_test.shape[0]

(785404, 196352)

In [9]:
n_books = df['book_id'].nunique()
print(f'Кол-во уникальных книг: {n_books}')

Кол-во уникальных книг: 10000


In [10]:
n_users = df['user_id'].nunique()
print(f'Кол-во уникальных пользователей: {n_users}')

Кол-во уникальных пользователей: 53424


В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

In [11]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(input_dim=n_books+1, output_dim=5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

Сначала мы задаём **размерность входного слоя (в этом параметре максимальное значение всегда равно длине вектора + 1)**. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. Далее мы разворачиваем результат в массив с одним измерением с помощью слоя `Flatten()`.

Делаем то же самое для пользователей:

In [12]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(input_dim=n_users+1, output_dim=5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

In [14]:
conc = Concatenate()([book_vec, user_vec])

Далее начинаем «собирать» нашу нейронную сеть из слоёв. `Dense` обозначает полносвязный слой. Также мы обозначаем для него **количество нейронов (на первом слое будет 128 нейронов, на втором - 32, на последнем (выходном) - 1)** и **данные (на первом слое принимаются данные от соединённых эмбеддингов, на втором - данные с первого слоя, а в последнем слое - данные со второго полносвязаного слоя)**, которые идут на вход.

In [15]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:



In [16]:
model2 = Model([user_input, book_input], out)

Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод **adam (одна из вариаций градиентного спуска)** и среднеквадратичную ошибку:

In [17]:
model2.compile(optimizer='adam', loss='mean_squared_error')

Теперь будем обучать нашу модель:



In [18]:
history = model2.fit([X_train['user_id'], X_train['book_id']], X_train['rating'], epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 42s 2ms/step - loss: 0.7973
Epoch 2/5
24544/24544 [==============================] - 40s 2ms/step - loss: 0.6865
Epoch 3/5
24544/24544 [==============================] - 41s 2ms/step - loss: 0.6558
Epoch 4/5
24544/24544 [==============================] - 41s 2ms/step - loss: 0.6315
Epoch 5/5
24544/24544 [==============================] - 40s 2ms/step - loss: 0.6093


В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети. На каждой из эпох обновляются веса для минимизации ошибки.

Теперь можно оценить качество:

In [21]:
model2.evaluate([X_test['user_id'], X_test['book_id']], X_test['rating'])

6136/6136 [==============================] - 4s 635us/step - loss: 0.7105


0.7105190753936768

К сожалению, результаты этого алгоритма нельзя зафиксировать стандартным `random_state`

Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [22]:
# Слои
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)
# Сборка и настройка модели
model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
# Обучение
result = model2.fit([X_train['user_id'], X_train['book_id']], X_train['rating'], epochs=10, verbose=1)
# Значение метрики MSE
model2.evaluate([X_test['user_id'], X_test['book_id']], X_test['rating'])

Epoch 1/10
24544/24544 [==============================] - 49s 2ms/step - loss: 0.6249
Epoch 2/10
24544/24544 [==============================] - 45s 2ms/step - loss: 0.5788
Epoch 3/10
24544/24544 [==============================] - 45s 2ms/step - loss: 0.5592
Epoch 4/10
24544/24544 [==============================] - 47s 2ms/step - loss: 0.5426
Epoch 5/10
24544/24544 [==============================] - 45s 2ms/step - loss: 0.5280
Epoch 6/10
24544/24544 [==============================] - 44s 2ms/step - loss: 0.5156
Epoch 7/10
24544/24544 [==============================] - 44s 2ms/step - loss: 0.5057
Epoch 8/10
24544/24544 [==============================] - 44s 2ms/step - loss: 0.4974
Epoch 9/10
24544/24544 [==============================] - 41s 2ms/step - loss: 0.4901
Epoch 10/10
6136/6136 [==============================] - 4s 640us/step - loss: 0.7743


0.7742751836776733